In [1]:
import os

In [19]:
#############################################################################################################
#   Create .config, .submit files -> change to your own folder
#
submit_dir = '/home/tchau/code/condor_submit/Significance'
logdir = '/scratch/tchau/DM_OscNext/MakePDF/logs/'
if not (os.path.exists(submit_dir)): os.makedirs(submit_dir)
if not (os.path.exists(logdir)): os.makedirs(logdir)


cfig_file = f'{submit_dir}/dagman.config'
sub_file = f'{submit_dir}/Sig.submit'
curdir=os.getcwd()
script = f'{curdir}/Significance.py'

with open(cfig_file, 'w') as c:
    c.write('DAGMAN_MAX_JOBS_SUBMITTED=100'+'\n')
    c.write('DAGMAN_MAX_SUBMIT_PER_INTERVAL=20'+'\n')
    c.write('DAGMAN_USER_LOG_SCAN_INTERVAL=10'+'\n')
    c.close()


with open(sub_file, 'w') as s:
    s.write(f'executable = {script}'+'\n')
    s.write('\n')
    s.write('initialdir = /data/user/tchau/'+'\n')
    s.write(f'logdir = {logdir}'+'\n')
    s.write('output = $(logdir)$(JOBNAME).$(Cluster).out'+'\n')
    s.write('error = $(logdir)$(JOBNAME).$(Cluster).err'+'\n')
    s.write('log = $(logdir)$(JOBNAME).$(Cluster).log'+'\n')
    s.write('\n')
    s.write('notification   = never'+'\n')
    s.write('universe       = vanilla'+'\n')
    s.write('\n')
    s.write('should_transfer_files = YES'+'\n')
    s.write('request_memory = 6GB'+'\n')
    s.write('getenv = True'+'\n')
    s.write('\n')
    s.write('arguments = -c $(ch) -p $(profile) --process $(prc) --outfile ($outfile)  --mass_default'+'\n')
    s.write('queue')
    s.close()

In [21]:
#############################################################################################################
#   Create .dag and submit script
#
submit = f'{submit_dir}/submit.sh'
dag_file = '{}/Significance.dag'.format(submit_dir)


#list of channels and profiles
# channels = ["numunumu"]
channels = ["WW","bb","tautau","mumu","nuenue", "numunumu", "nutaunutau"]
profile = ['NFW','Burkert']

# process
prcs = ["ann", "decay"]

# number of trials for TS distribution
ntrials=10000

# Loops on scenarios and submit jobs for each of them
with open(dag_file, 'w') as f:
    for c in channels:
        for p in profile:
            for prc in prcs:
                # output
                outfile=f"/data/user/tchau/DarkMatter_OscNext/Significance/{c}_{p}_{prc}"
                f.write('JOB significance_{}_{}_{}'.format(prc, c, p)+' Sig.submit \n')
                f.write('VARS significance_{}_{}_{}'.format(prc, c, p)
                        +' JOBNAME="significance_{}_{}_{}"'.format(prc, c, p) 
                        +' ch="{}" profile="{}" prc="{}" outfile="{}"'.format(c, p, prc, outfile)+'\n')
f.close()
with open(submit, 'w') as s:
    s.write("#!/bin/sh\n")
    s.write("condor_submit_dag -config dagman.config {}\n".format(dag_file))
    print("condor_submit_dag -config dagman.config {}\n".format(dag_file))
s.close()


condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/Significance/Significance2.dag

